# A brief introduction to machine learning in environmental science

https://github.com/wkearn/evsc4002-machine-learning

The goal of this interactive presentation is to demonstrate how machine learning can help us understand and predict environmental processes. We'll work through a problem that comes from my own research and see how we can use a few particular algorithms from the machine learning world to solve it. Along the way, I will emphasize the practical aspects of machine learning. You probably won't ever work on the exact problem that I will show you, or use the exact same tools, but there are some fundamental ideas to creating models, fitting them and testing them that will carry over if you do find a problem that you want to tackle with machine learning.

This interactive notebook will always be available to you at https://github.com/wkearn/evsc4002-machine-learning. Feel free to send me an email ([wkearn@virginia.edu](mailto:wkearn@virginia.edu)) or stop by my office (Clark 357) if you have any more questions!

## Outline

0. Using the Jupyter notebook
1. A simple example: unsupervised classification
2. The problem of estimating tidal flows from water levels
3. Analyzing model performance
4. A classical semi-empirical model
5. A linear time series model
6. A nonlinear time series model

## Jupyter and Julia

This [Jupyter](https://jupyter.org/) notebook provides an interactive environment that we'll use to run code that will train machine learning models on some data that I will provide to you. The notebook allows you to run code and visualize the results immediately.

Jupyter notebooks are divided into cells. If you click on this text, you'll see a blue bar to the left that highlight's the current cell. There are two kinds of cells. This cell is a Markdown cell, which contains text written in the Markdown format. 

The next cell is a code cell, which contains code that can be run.

In [ ]:
# This is a code cell. 
# Highlight this cell and press `Shift-Enter` 
# to execute the code within the cell
1+1

To edit a code cell, click on it. To edit a Markdown cell, double click, and you will be able to type in the cell. If you are in editing mode, `Shift-Enter` will still execute the code and move you to the next cell. Press `ESC` or click on another cell to get out of the editing mode without executing the cell.

To change the type of a cell from Markdown to code or vice versa, you can highlight the cell and then use the dropdown menu on the toolbar. You can also use a keyboard shortcut from the highlight mode (they won't work if you are in editing mode. Press `ESC` to get out of editing mode then use the shortcut):

- `y` will convert a Markdown cell to a code cell
- `m` will convert a code cell to a Markdown cell 

Jupyter notebooks can run code written in a variety of programming languages including Python and R. This notebook runs code in the [Julia](http://julialang.org/) programming language, which I use for all of my research. Don't worry if you've never programmed before or if you have, but you don't have any experience with Julia. I've written the examples so that you can run them without writing any code. Some of the examples will have parameters that you can play with to see how they change your output, but those will be clearly marked and will usually just require you to change a number in the code.

In [ ]:
# This code tests whether the expression on the right 
# equals that on the left.
# Change the value on the right to make the test 
# return `true` and execute the cell to check your work.

1 + 1 == 3

We'll need a few packages to load our data, train our models and visualize our results. Julia is designed to be a really fast language, but one thing that can take a while is loading packages. We'll load these packages while we move on to the next part.

In [ ]:
using Plots, Clustering, LinearAlgebra, NetCDF, 
DelimitedFiles,  StatsBase, Statistics, Optim,
FileIO, Random
import Distances: pairwise, Euclidean
gr(format=:png)
theme(:wong)
Random.seed!(1234)

If at any point you get an error that says something like `UndefVarError: f not defined`, you may have forgotten to run one of the code cells. Go back and make sure that you've run every cell above the one that failed that has some code in it.

## A simple example: unsupervised classification

Machine learning is the art and science of training computers to recognize patterns in data and to apply those patterns to accomplish a given task. An example of these tasks that is commonly studied in machine learning is **unsupervised classification**, trying to separate data into natural clusters.

### Unsupervised learning

- Extracting patterns from data that has not been **labeled** or associated with an answer already

### Supervised learning

- Learning to map inputs to outputs based on example input-output pairs

### Classification

- Learning to segment a data set into a discrete number of classes.

### Regression

- Learning to associate input data with a continuous output

Imagine we have data that looks like this:

In [ ]:
U = rand(1000,2)
V = rand(1000,2)
R = U.+[0,2]'
θ = 2pi*V
X = sqrt.(R).*cos.(θ)
Y = sqrt.(R).*sin.(θ)
scatter(X,Y,c=:grey,leg=false,xlabel="X",ylabel="Y")

### Observation

- A single measurement of one object of interest

### Feature

- A measured property of each object

We can see that these data points separate naturally into two clusters based on their distance from the origin. The unsupervised classification task is to label each point with its cluster membership. 

One algorithm that is commonly applied to this task is **k-means clustering** ([Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering)), which defines each cluster by a *centroid*, a point in the data space. Each data point is assigned to the cluster whose centroid is closest to it. The k-means algorithm iteratively adjusts the location of these centroids until it finds the locations where the total distance between each of the points and its assigned centroid is minimized.

Julia's [Clustering.jl](https://clusteringjl.readthedocs.io/en/latest/kmeans.html) package has an implementation of k-means clustering that we can use.

### k-means clustering

- An unsupervised classification algorithm that splits data into $k$ clusters defined by the mean of all the observations in that cluster (called the **centroid**)
- The algorithm adjusts the cluster assignments to minimize the sum of the distances from all of the points in each cluster and their corresponding centroid.

In [ ]:
# We need a matrix that holds each sample in an individual column
D = [vec(X)'; vec(Y)'] 

# The first argument to the `kmeans` function is the data matrix, 
# the second is the number of clusters
kmeans_result = kmeans(D,2) 

We can extract the cluster assignments from the k-means result and plot them.

In [ ]:
kmeans_assignments = assignments(kmeans_result)
p = plot(leg=false)
scatter!(D[1,kmeans_assignments.==1],D[2,kmeans_assignments.==1])
scatter!(D[1,kmeans_assignments.==2],D[2,kmeans_assignments.==2])


That doesn't seem to have worked. Why not?

### Linear model

- A model that relies on a **linear combination** of features to achieve its goal

$$
\hat{y}_i = \sum_{j=1}^J \beta_j x_{ij}
$$

### Nonlinear model

- Any model that is not a linear one.

We can use another tool (often called the **kernel trick**) to cluster these data correctly.

The details aren't super important, but we're basically going to run two transformations on our data:

1. Compute a nonlinear kernel that encodes our knowledge of how the clusters should separate in space.
2. Apply something like principal components analysis (which you may have encountered before in a stats class) to the kernel to transform our data into a new space in which they are linearly separable, so k-means will work.

This technique is called [spectral clustering](https://www.cs.cmu.edu/~aarti/Class/10701/readings/Luxburg06_TR.pdf).

### Kernel trick

- To convert a linear algorithm into a nonlinear one, apply a **kernel function** to the data.
- This kernel function transforms the data into a new geometric space in which the linear algorithm will work

In [ ]:
# This is our kernel function, a Gaussian radial basis function
k(h,σ=1.0) = exp.(-σ*h.^2)

# We first calculate a distance matrix, 
# then we apply our kernel function to get
# a kernel (or Gram) matrix
H = pairwise(Euclidean(),D)
K = k.(H,100)

# We have to normalize it in a particular way.
D2 = diagm(0=>vec(1.0./sqrt.(sum(K,dims=2))))
L = Symmetric(D2*K*D2)

# This is some linear algebra that computes 
# the principal components analysis
l,Z = eigen(L);

# Since we are looking for two clusters, we'll
# take the first two principal components as our
# new data set. 
Y = Z[:,end-1:end]

Let's look at the data in the transformed space.

In [ ]:
scatter(Y[:,1],Y[:,2],c=:grey,leg=false)

In [ ]:
# Finally, we create a transformed data matrix 
# and run k-means on the transformed data.
D2 = Matrix(Y')
kmeans2 = kmeans(D2,2);

In [ ]:
kmeans_assignments2 = assignments(kmeans2)
p = plot(leg=false)
scatter!(D2[1,kmeans_assignments2.==1],D2[2,kmeans_assignments2.==1])
scatter!(D2[1,kmeans_assignments2.==2],D2[2,kmeans_assignments2.==2])

In [ ]:
p = plot(leg=false)
scatter!(D[1,kmeans_assignments2.==1],D[2,kmeans_assignments2.==1])
scatter!(D[1,kmeans_assignments2.==2],D[2,kmeans_assignments2.==2])

The details of this example are not particularly important because you are rarely going to see a problem like this one in real life. What is important is to realize that machine learning boils down to **pattern recognition**. Our human perception has evolved over millions of years to be good at this task: we can clearly see even from the unlabeled data that there are two clusters in these data, depending on their radial distance from the origin. But computers are not so sophisticated. Simple algorithms, like k-means, are powerful when data meet their assumptions, but we have to be careful with more complicated datasets. 

We should never simply apply a common machine learning method to our data. Instead, we test it to figure out when it works and when we need to add more complexity to our model by, for example, applying the kernel trick to make a linear model nonlinear.

# Estimating tidal flows from water levels

![](figures/sweeney.png)

One interesting way to study salt marshes is to look at the flux of material into and out of the marsh. For instance, marshes need a steady supply of sediment to accrete as sea level rises, so if we measure the amount of sediment that flows into a marsh on the flood tide and the amount that comes out on the ebb tide, we can determine whether a marsh is gaining or losing sediment (and thus elevation relative to sea level).

However, it can be tricky to make good measurements of the flow of water in marsh creeks, so one question is whether we can use something that is easier to measure, like the water level, to estimate what the flow is.

What we need is a model that relates water level (also called **stage**) to flow (also called **discharge**, a **stage-discharge model** or **rating curve**. Building these rating curves is how [the USGS estimates discharges](https://maps.waterdata.usgs.gov/mapper/index.html) in streams across the country. For example, here is a rating curve used by the USGS to estimate [discharges on the Rivanna River at Palmyra](https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=02034000).

In [ ]:
palmyra_data = readdlm("data/rating_data.csv",header=true)[1]
palmyra_curve = readdlm("data/rating.csv",header=true)[1]
palmyra_h = float.(palmyra_curve[:,1])
palmyra_q = float.(palmyra_curve[:,3])
scatter(palmyra_data[:,2],palmyra_data[:,1])
plot!(palmyra_q,palmyra_h,c=:black,leg=false,
    ylabel="Stage (ft)",xlabel="Discharge (ft³/s)")

The USGS deploys a water level logger in the stream and records the stage. Then they use this rating curve to estimate the discharge. When you download discharge data from their website, you are almost always getting this estimated discharge, not measured discharge.

We'd like to build a rating curve for our salt marsh stage and discharge data so that we can use a water level logger to estimate discharge. Let's take a look at some real data to see what we are working with.

In [ ]:
# This function will load some stage and discharge data for us
# if we give it a creek and a deployment number (more on the different deployments later)

function load_data(creek,deployment)
    H = ncread(joinpath("data",creek,creek*lpad(deployment,2,'0')*".nc"),"H")
    Q = ncread(joinpath("data",creek,creek*lpad(deployment,2,'0')*".nc"),"Q")
    
    H,Q    
end

In [ ]:
# We'll load data from Sweeney Creek
H,Q = load_data("sweeney",2)

# And plot it up
plot(layout=grid(3,1,heights=[0.2;0.2;0.6]),
    plot(H,ylabel="H (m)",c=:black,leg=false),
    plot(Q,ylabel="Q (m²/s)",c=:black,leg=false),
    plot(Q,H,c=:black,leg=false, xlabel="Specific discharge (m²/s)",ylabel="Stage (m)"),
    size=(500,500)
    )

Note that the stage-discharge plot looks very different from the data in a river. First, there are flows in two directions, positive discharges on the flood tide and negative discharges on the ebb tide. Also, the fortnightly spring-neap tidal cycle means that one water level corresponds to many different discharges, depending on how fast the tide is rising.

The USGS rating curve is a nonlinear function, but it is ultimately just a fancy kind of linear regression. What happens if we fit a linear regression to our stage-discharge data?

In [ ]:
# This code fits a linear regression of order k to the data (X,Y)
struct PolynomialRegression
    X
    Y
    H
    β
    k
end

function StatsBase.fit(::Type{PolynomialRegression},X,Y,k=1)
    H = hcat([X.^i for i in 0:k]...)
    β = H\Y
    PolynomialRegression(X,Y,H,β,k)
end
StatsBase.predict(m::PolynomialRegression) = m.H*m.β
StatsBase.predict(m::PolynomialRegression,x) = hcat([x.^i for i in 0:m.k]...)*m.β

In [ ]:
m1 = fit(PolynomialRegression,H,Q,1)

In [ ]:
plot(Q,H,
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(m1,sort(H)),sort(H),c=:black,
    title="Linear regression")

Not surprisingly, the linear regression draws a line through the data points. But because there are both flood and ebb discharges, we get a vertical line pretty close to zero. This is one of the major challenges of rating curve development in tidal environments: we can always estimate a discharge of zero, and we won't be that far away from the true discharge, but such a model is pretty useless if we want to estimate fluxes of sediment or nutrients into the marsh.

We know that rating curves in general are nonlinear, look at the Palmyra curve, for example. We tried a first-order linear regression above, but what if we let it be quadratic or cubic? That's easy enough to try: when you see the line 

```julia
fit(PolynomialRegression,H,Q,1)
```

that `1` is the highest order of the polynomial regression. Try changing it to another positive integer and see what you can make happen. Remember click in the code cell to edit it, `Shift-Enter` to run the cell that fits the model, then `Shift-Enter` again to run the cell that makes the plot.

Congratulations! You've just fit your first stage-discharge model. Linear regression may seem pretty mundane, but it is very closely related to other regression algorithms used in machine learning, but significantly cheaper to fit. When in doubt, try a linear regression!

## Analyzing the performance of a model

We can see from the plot that the model doesn't look like the data, but how do we get a quantitative estimate of model performance that lets us compare different models?

There are many different ways to calculate model performance, but we will use a metric common in hydrology called the **Nash-Sutcliffe efficiency** (NSE). 

We noted above that estimating discharge with zero (or something like the mean discharge) is not all that bad. So the NSE quantifies how well a given model performs relative to the constant model.

#### Nash-Sutcliffe Efficiency

$$
NSE = 1 - \frac{\sum_{t=1}^T (Q_m^t - Q_o^t)^2}{\sum_{t=1}^T (Q_o^t - \overline{Q_o})^2}
$$

where now $Q_m^t$ is the modeled discharge at time $t$, $Q_o^t$ is the observed discharge at time $t$, and $\overline{Q_o}$ is the mean of the discharge over the time series.

In [ ]:
function nse(Qo,Qm)
   1. -sum(abs2,Qm-Qo)./sum(abs2,Qo.-mean(Qo)) 
end

NSE is quite similar to $R^2$, which you may have used before, except that it can range from $-\infty$ to 1. An NSE of 1 is a perfect fit. An NSE of 0 means that you get the same mean squared error as estimating the discharge using the mean. A negative NSE, means you're doing worse than estimating with the mean. This can happen when you're using complicated models, and we will see some examples of negative NSE later.

When we compute the NSE for the linear regression, we see that it is pretty close to zero. The linear regression model is not much better than more or less using the mean of the data (because a linear regression in a tidal system is just using the mean of the data).

In [ ]:
nse(Q,predict(m1,H))

# A classical semi-empirical tidal stage-discharge model

It is clear from our analysis above that we'll need to go beyond a linear regression to come up with a good stage-discharge model. One approach that was taken by John Boon in the 1970s, is to approximate the equations of motion governing flow in shallow water. We won't go through the derivation, because it is easy to understand intuitively.

The main assumption made by Boon is that the water surface everywhere in our marsh is flat.

![](figures/boonModel.png)

As the tide rises, enough water flows into it to fill up the entire marsh to the new water level.

![](figures/boonModel2.png)

And the process is reversed as the tide falls. This gives us a model for the water level and discharge

$$
Q(t) = A(H)\frac{dH}{dt}
$$

where $A(H)$ is the **hypsometric curve**, which tells you how much area you need to fill in to bring the water level to $H$. The hypsometric curve depends on the topography of the marsh, which we could, in principle, estimate using remote sensing data. But we don't need to do that, because we can use our stage-discharge data to estimate the function $A(H)$.

For completeness, our model for discharge is

$$
\hat{Q}_i = \alpha |H_i|^{\beta} \cdot (H_i-H_{i-1})
$$

where $\alpha$ and $\beta$ are the coefficients that we fit. We have approximated the function $A(H)$ as a power law, because power law hypsometric curves are fairly good approximations of what we see in nature, and it keeps the computation tractable. We have approximated the time derivative $dH/dt \approx H_i - H_{i-1}$. This means that we won't be able to estimate the discharge at time step $i=1$, because we don't have $H_0$, so we'll fit the model by minimizing the mean squared error between the modeled and observed discharges for the time steps $i \in \{2 \dots N\}$

In [ ]:
struct BoonModel
    β
    H
    Q
end

boon(h,β) = β[1]*abs(h[1]).^β[2]*(h[2]-h[1])
function boon(H::AbstractMatrix,β)
    Q = zeros(size(H,1))
    for i in 1:length(Q)
        Q[i] = boon(H[i,:],β)
    end
    Q
end
ssr(H,Q,β) = 0.5*msd(Q,boon(H,β))

function StatsBase.fit(::Type{BoonModel},H,Q)
    Hm = [H[1:end-1] H[2:end]]
    opt = optimize(β->ssr(Hm,Q[2:end],β),zeros(2))
    β = Optim.minimizer(opt)
    BoonModel(β,H,Q)
end

StatsBase.predict(m::BoonModel) = predict(m,m.H)
StatsBase.predict(m::BoonModel,H) = boon([H[1:end-1] H[2:end]],m.β)

In [ ]:
m2 = fit(BoonModel,H,Q)

In [ ]:
plot(Q[2:end],H[2:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(m2,H),H[2:end],c=:black,title="Boon model")
# Add the NSE to the plot
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Q[2:end],predict(m2,H)),digits=3))")

Our Nash-Sutcliffe efficiency is up to 0.86! That's pretty good, but we can see from the plot that we don't do a great job of estimating the asymmetry between flood and ebb tides. That asymmetry is especially important if we want to get a good estimate of the balance between sediment being brought in on the flood tide and brought out on the ebb tide.

# A linear time series model

### Lagged regression

The Boon model takes the form

$$
Q_i = A(H_i) \cdot (H_i - H_{i-1}) = \alpha |H_i|^\beta \cdot H_i - \alpha H_i^\beta \cdot H_{i-1}
$$

This is effectively a regression like we did above. There are two differences:

- We use values of the water level at previous time steps (**lagged values**) instead of higher powers of the water level at the current time step
- The regression coefficient is $\alpha H_i^\beta$, which is a nonlinear function of $\alpha$ and $\beta$, so that this is a **nonlinear regression**

From this perspective, we can see two ways that we might make the Boon model a little more flexible, letting it fit a more complex stage-discharge relationship.

- We could look further back in time
- We could make it more nonlinear

Let's first examine the first option. Looking further back in time might help us account for the phase lag between stage and discharge that causes the tidal asymmetry. The reasons for this rely on some dynamical systems theory that is pretty complicated, but you don't have to take my word for it, let's try it out!

We'll use a linear time invariant model that is commonly used in signal processing. This is just a multiple linear regression on lagged values of stage. It comes with one parameter that you can tune: the number of lags that you want to consider. 

In [ ]:
# This code will fit a linear time invariant model
struct LinearTimeInvariant
    n
    β
    H
    Q
end

function StatsBase.fit(::Type{LinearTimeInvariant},H,Q,n=1)
    N = length(H)
    Hm = zeros(N-n+1,n)
    for i in 1:n
        Hm[:,i] = H[i:N-(n-i)] 
    end
    β = Hm\Q[n:end]
    LinearTimeInvariant(n,β,H,Q)
end

function StatsBase.predict(m::LinearTimeInvariant,H)
    N = length(H)
    Hm = zeros(N-m.n+1,m.n)
    for i in 1:m.n
        Hm[:,i] = H[i:N-(m.n-i)] 
    end
    Hm*m.β
end

In [ ]:
m3 = fit(LinearTimeInvariant,H,Q,75)

In [ ]:
plot(Q[m3.n:end],H[m3.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(m3,H),H[m3.n:end],c=:black,
    title="Linear time invariant model")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Q[m3.n:end],predict(m3,H)),digits=3))")

That doesn't seem to work quite as well as the Boon model, but we do get a little more asymmetry between flood and ebb.

## Choosing the number of lags

I gave you a starting value for the number of lags of 75. These data are recorded at 10 minute intervals, so 75 time steps is approximately one tidal cycle. Try increasing the number of lags. How does the performance change?

**Warning**: Don't try lag of more than 2000 or so, it can break things in the rest of the notebook.

In [ ]:
m4 = fit(LinearTimeInvariant,H,Q,75)
plot(Q[m4.n:end],H[m4.n:end],
    xlims=(-2.0,1.5),
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(m4,H),H[m4.n:end],c=:black)
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Q[m4.n:end],predict(m4,H)),digits=3))")

That does a pretty good job, right? 

It does a pretty good job **on the training data**, the data that we have used to fit the model. With most machine learning models, you'll have a tuning parameter that controls how many parameters the model fits. Here we have one parameter per lag. Generally you'll be able to fit more and more complicated functions with more and more parameters. So why don't we always use as many parameters as we can? Let's try to apply our model to some new data from the same marsh and see what happens.

### Training data

- Data that you have used to fit the model

### Test data

- Data that comes from the same source as your training data, but that you have held out from your fitting

In [ ]:
Ht,Qt = load_data("sweeney",4)

In [ ]:
plot(Q,H,
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(Qt,Ht,c=:black,
    title="Grey = Deployment 2 data; Black = Deployment 4 data")

We can see that the shape of the stage-discharge relationship is pretty consistent across deployments in the same marsh. So our giant linear time invariant model that works great on the training data should work well on the new **test data**, right?

In [ ]:
m5 = fit(LinearTimeInvariant,H,Q,2000)
plot(Qt[m5.n:end],Ht[m5.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(m5,Ht),Ht[m5.n:end],c=:black,title="Linear time invariant model, test data")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Qt[m5.n:end],predict(m5,Ht)),digits=3))")

### Cross-validation

- Fitting a model on training data and then calculating the performance of the model by applying it to test data

The sharp decline in model performance from the training data to the test data is a sign of **overfitting**. We can fit the training data arbitrarily closely by increasing the number of parameters, but we end up fitting the model to unique features of the training data that don't show up in the test data. The common example given for overfitting is trying to fit a high-order polynomial regression to a few data points:

### Overfitting

In [ ]:
x = range(0,stop=1,length=10)
xt = range(0,stop=1,length=100)
y = x.^2 + 0.1*randn(10)
scatter(x,y,leg=false,c=:grey)
m10 = fit(PolynomialRegression,x,y,10)
plot!(x->x^2,c=:black)
plot!(xt,predict(m10,xt))
annotate!(0.25,1.0,"Training NSE: "*"$(round(nse(y,predict(m10,x)),digits=3))")

Here the true function is in black. The overfit function is in green. It has a training NSE of 1.0 because it interpolates perfectly through the data, but if you got more data from the same system (falling along the black line), you would do much worse. The same thing happens when we fit a really large linear time invariant model to our stage-discharge data.

We can avoid overfitting by choosing the number of lags in our linear time invariant model with cross validation. We fit the model to the training data with a bunch of different options for the lag parameter, then compute the NSE on the test data. We pick the model with the highest NSE and use it in the future.

If we do that for the Sweeney Creek stage-discharge data, we find that the optimal number of lags is around 900.

Before we leave the linear time invariant model, what do the predictions from that optimal model (with 900 lags) look like?

In [ ]:
lti_900 = fit(LinearTimeInvariant,H,Q,900)
plot(Qt[lti_900.n:end],Ht[lti_900.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(lti_900,Ht),Ht[lti_900.n:end],c=:black,
    title="Optimal linear time invariant model")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Qt[lti_900.n:end],predict(lti_900,Ht)),digits=3))")

The Nash-Sutcliffe efficiency is 0.808, which is worse than the Boon model. Adding more memory helps us fix the asymmetry, but we lose some of our predictive ability (i.e. the NSE goes down).

# A nonlinear time series model

When we were looking at extensions of the Boon model, I mentioned that there were two ways to extend it. Either we could add more memory to the model, hoping to correct the phase lags that we see in the Boon model, or we could add more nonlinearity to the model, because we know from the physics that nonlinearities in tidal propagation generate the phase lags.

Remember the kernel trick that allowed us to correctly classify the bullseye data set?

In [ ]:
p = plot(leg=false)
scatter!(D[1,kmeans_assignments2.==1],D[2,kmeans_assignments2.==1])
scatter!(D[1,kmeans_assignments2.==2],D[2,kmeans_assignments2.==2])

We can do the same thing to our linear time-invariant model to create a nonlinear time-invariant model. We could use the Gaussian kernel that we used above, but an inhomogeneous polynomial kernel

$$
k_p(x,y) = (x^Ty + 1)^p
$$

lets us interpret this particular model as a [Volterra series](https://en.wikipedia.org/wiki/Volterra_series), which is a commonly used nonlinear extension for linear time series models (Franz and Schölkopf 2006). However, it is important to remember that there isn't really anything special about time series. All we are doing is a nonlinear regression using our kernel trick.

In [ ]:
# This code will fit a Volterra series model
struct RegularizedVolterra
    n
    λ
    p
    β
    H
    Q
end

# This is our kernel function, written two ways.
k2(x,y,p=2) = (x'y+1.0).^p
k2(d,p=2) = (d+1).^p

function kernel_matrix(H1,H2,n,p)
    N = length(H1)
    M = length(H2)
    H1m = zeros(N-n+1,n)
    H2m = zeros(M-n+1,n)

    for i in 1:n
        H1m[:,i] = H1[i:N-(n-i)] 
        H2m[:,i] = H2[i:M-(n-i)] 
    end
    
    k2.(H1m*H2m',p)
end

kernel_matrix(H,n,p) = kernel_matrix(H,H,n,p)

kernel_matrix(m::RegularizedVolterra,H) = kernel_matrix(m.H,H,m.n,m.p)

function StatsBase.fit(::Type{RegularizedVolterra},H,Q,n=1,p=1,λ=0.0)
    N = length(H)
    β = (kernel_matrix(H,n,p) + λ*I)\Q[n:end]
    RegularizedVolterra(n,λ,p,β,H,Q)
end

function StatsBase.predict(m::RegularizedVolterra,H)
    N = length(H)
    Hm = zeros(N-m.n+1,m.n)
    for i in 1:m.n
        Hm[:,i] = H[i:N-(m.n-i)] 
    end
    kernel_matrix(m,H)'m.β
end

We now have two parameters, the number of lags, $n$, and the order of the nonlinearity, $p$. Not surprisingly, we can choose $p$ with cross-validation the same way we chose $n$ in the linear, time-invariant model. If we do that, we find that the optimal model order is around 3.

In [ ]:
mV1 = fit(RegularizedVolterra,H,Q,75,3,0.0)
plot(Qt[mV1.n:end],Ht[mV1.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(mV1,Ht),Ht[mV1.n:end],c=:black,title="Volterra model")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Qt[mV1.n:end],predict(mV1,Ht)),digits=3))")

It seems to get the asymmetry and magnitudes of the discharges right, but it seems really noisy, right?

This noise is another symptom of overfitting. To check, see how well the same model does on the training data:

In [ ]:
plot(Q[mV1.n:end],H[mV1.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(mV1,H),H[mV1.n:end],c=:black,title="Volterra model: training data")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Q[mV1.n:end],predict(mV1,H)),digits=3))")

That's basically perfect! So the noisiness on the test data must come down to overfitting. We have a few options to play with, here. We could reduce the number of lags or the order of the nonlinearity, but that won't help us very much in the end, and it might even make things worse (we have chosen our nonlinearity to optimize the cross validated NSE, after all).

## Regularization

The solution is to use **regularization**, which is a technique to prevent overfitting by forcing your algorithm to prefer smoother functions. Regularization introduces an extra parameter that controls the tradeoff between fitting the data and a smoother estimate. A higher setting of this parameter leads to smoother functions but will fit the data worse. Obviously there is an optimum, and we can use cross validation to find it.

To see how regularization works, let's play with the overfit polynomial that we saw earlier.

In [ ]:
scatter(x,y,leg=false,c=:grey)
plot!(x->x^2,c=:black)
plot!(xt,predict(m10,xt))

In [ ]:
struct RegularizedPolynomialRegression
    X
    Y
    H
    β
    λ
    k
end

function StatsBase.fit(::Type{RegularizedPolynomialRegression},X,Y,k=1,λ=0)
    H = hcat([X.^i for i in 0:k]...)
    β = (H'H +λ*I)\(H'Y)
    RegularizedPolynomialRegression(X,Y,H,β,λ,k)
end
StatsBase.predict(m::RegularizedPolynomialRegression) = m.H*m.β
StatsBase.predict(m::RegularizedPolynomialRegression,x) = hcat([x.^i for i in 0:m.k]...)*m.β

Now we can play around with the regularization parameter. Try a few values out an see what happens. Remember that the regularization parameter is always greater than or equal to 0, and a value of 0 means no regularization (i.e. identical to the model that we fit above). 

If we really wanted to choose the "right" value of the regularization parameter, we would, of course, cross validate.

In [ ]:
# The regularization parameter is the last number on the next line.
m10r = fit(RegularizedPolynomialRegression,x,y,10,0.0)

scatter(x,y,leg=false,c=:grey)
plot!(x->x^2,c=:black)
plot!(xt,predict(m10r,xt))

Remember that we can prevent overfitting in this polynomial regression by using a smaller order of the polynomial, but unless our data actually follow a low-order polynomial, we may not fit our data particularly well.

Regularization lets us use a really flexible model that is capable of capturing all of the interesting behavior in the data but also control its tendency to overfit.

Since the Volterra series model is just a linear regression with the kernel trick applied, we can add a regularization parameter exactly the same way.

You can play with adjusting the regularization parameter here.

In [ ]:
# The regularization parameter is the last number on the next line.
mVr = fit(RegularizedVolterra,H,Q,75,3,0.0)
plot(Qt[mVr.n:end],Ht[mVr.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(mVr,Ht),Ht[mVr.n:end],c=:black,title="Regularized Volterra model")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Qt[mVr.n:end],predict(mVr,Ht)),digits=3))")

As you may have guessed, we can choose the regularization parameter by cross-validation. When we do that, we find that the optimal model order is $p=3$ and the optimal regularization parameter is $\lambda=100$.

In [ ]:
mVr_CV = fit(RegularizedVolterra,H,Q,75,3,100.0)
plot(Qt[mVr_CV.n:end],Ht[mVr_CV.n:end],
    c=:grey,leg=false,
    xlabel="Specific discharge (m²/s)",
    ylabel="Stage (m)")
plot!(predict(mVr_CV,Ht),Ht[mVr_CV.n:end],c=:black,title="Cross-validated regularized Volterra model")
annotate!(1.0,0.5,"NSE: "*"$(round(nse(Qt[mVr_CV.n:end],predict(mVr_CV,Ht)),digits=3))")

# Conclusions

- Machine learning is all about fitting statistical models to data

- Tools like **the kernel trick**, **cross validation** and **regularization** let us build far more complicated models than classical statistics provides.

- The stage-discharge modeling problem illustrates one route to solving an environmental data analysis problem:

   - Start with simple models like a linear regression or the Boon model.
   - Add complexity by looking at different features (more lags) or more complex models (more nonlinearity)
   - Test models with cross validation
   - Regularize

- Machine learning can help us out with lots of other environmental problems
    - When physically-based models are hard to get right
    - When we have a lot of data
        - Oceanographic time series
        - Remote sensing
        - Eddy flux

# Resources

There is a lot of stuff out there about machine learning and Googling for a little bit will turn up a lot of great resources if you would like to learn more.

[scikit-learn](https://scikit-learn.org/stable/index.html) is a Python package for machine learning. Their documentation is great and covers a lot of different algorithms. 

A lot of what I know about building and testing complex models comes from Cosma Shalizi's book [*Advanced Data Analysis from an Elementary Point of View*](http://www.stat.cmu.edu/~cshalizi/ADAfaEPoV/ADAfaEPoV.pdf), but it does requires a solid understanding of basic statistics, particularly linear regression.

If you want to learn more about stage-discharge models, check out this paper:

Kearney, W. S., Mariotti, G., Deegan, L. A. and Fagherazzi, S. 2017. Stage-discharge relationship in tidal channels. *Limnology and Oceanography: Methods*. https://doi.org/10.1002/lom3.10168 

The seminal paper on discharge asymmetry by John Boon is

Boon, J. D. Tidal discharge asymmetry in a salt marsh drainage system. 1975. *Limnology and Oceanography*. https://doi.org/10.4319/lo.1975.20.1.0071

The Volterra series model and its links to kernel regression are explored in

Franz, M. O. and Schölkopf, B. 2006. A unifying view of Wiener and Volterra theory and kernel polynomial regression. *Neural Computation*. https://doi.org/10.1162/neco.2006.18.12.3097